In [2]:
import pandas as pd
import ast
import streamlit as st
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df_filtered = pd.read_csv('../data/all_pages.csv')

# コラム厳選
def to_dict(value):
    if value is None:
        return None
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None

def to_list(value):
    if value is None:
        return None
    if isinstance(value, list):
        return value
    if isinstance(value, str):
        try:
            return ast.literal_eval(value)
        except Exception:
            return None
    if isinstance(value, float) and pd.isna(value):
        return None
    return None


# -----------------
# データ整形
# -----------------
if 'client' in df_filtered.columns:
    df_filtered['client_dict'] = df_filtered['client'].apply(to_dict)
    df_filtered['client_name'] = [d.get('name') for d in df_filtered['client_dict']]
    df_filtered['employee_count'] = [d.get('employee_count') for d in df_filtered['client_dict']]
    df_filtered['established'] = [d.get('established_at') for d in df_filtered['client_dict']]
    # df_filtered['employee_count'] = df_filtered['client_dict'].apply(lambda d: d.get('employee_count') if isinstance(d, dict) else None)
    # df_filtered['established'] = df_filtered['client_dict'].apply(lambda d: d.get('established_at') if isinstance(d, dict) else None)
    df_filtered['established'] = pd.to_datetime(df_filtered['established'], unit='s')
    df_filtered = df_filtered.drop(['client', 'client_dict'], axis=1, errors='ignore')

if 'job_offer_skill_names' in df_filtered.columns:
    df_filtered['job_offer_skill_names'] = df_filtered['job_offer_skill_names'].apply(to_list)

if 'job_offer_areas' in df_filtered.columns:
    df_filtered['job_offer_areas'] = df_filtered['job_offer_areas'].apply(to_list)

if 'job_offer_name' in df_filtered.columns:
    df_filtered['job_tag'] = 'その他'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア|データ分析基盤エンジニア|DBエンジニア'), 'job_tag'] = 'データエンジニア'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'
    df_filtered.loc[df_filtered['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア|LLMエンジニア|MLエンジニア'), 'job_tag'] = 'AIエンジニア'

# 欠損値中央値補完
if 'job_offer_max_salary' in df_filtered.columns:
    median_salary = df_filtered['job_offer_max_salary'].median()
    df_filtered['job_offer_max_salary'] = df_filtered['job_offer_max_salary'].replace(0, np.nan)
    df_filtered['job_offer_max_salary'] = df_filtered['job_offer_max_salary'].fillna(median_salary)

if 'job_offer_max_salary' in df_filtered.columns and 'job_offer_min_salary' in df_filtered.columns:
    df_filtered['avg_salary'] = (df_filtered['job_offer_max_salary'] + df_filtered['job_offer_min_salary'])/2

new_order = [
    'job_offer_id',
    'client_name',
    'job_offer_name',
    'job_tag',
    'job_offer_areas',
    'job_offer_skill_names',
    'employee_count',
    'established',
    'job_offer_min_salary',
    'job_offer_max_salary',
    'avg_salary',
]
df_filtered = df_filtered[new_order]

In [9]:
df_filtered.isnull().sum()

job_offer_id              0
client_name               0
job_offer_name            0
job_tag                   0
job_offer_areas           0
job_offer_skill_names     0
employee_count            0
established              16
job_offer_min_salary      0
job_offer_max_salary      0
avg_salary                0
dtype: int64

In [29]:
df_filtered.columns.value_counts

<bound method IndexOpsMixin.value_counts of Index(['job_offer_id', 'job_offer_name', 'job_offer_areas',
       'job_offer_min_salary', 'job_offer_max_salary', 'job_offer_skill_names',
       'client_name', 'employee_count', 'established', 'job_tag'],
      dtype='object')>

In [30]:
df_filtered[df_filtered['job_tag']=='データエンジニア']


,job_offer_id,job_offer_name,job_offer_areas,job_offer_min_salary,job_offer_max_salary,job_offer_skill_names,client_name,employee_count,established,job_tag
7,294081,データエンジニア（メンバーポジション）,"[SQL, Python, AWS, Azure, GCP]",400,550,"[SQL, Python, AWS, Azure, GCP]",株式会社 cococo,10,2023-11-30 15:00:00,データエンジニア
12,187679,（職種未経験者歓迎）データエンジニア,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",400,600,"[C, SQL, Java, Python, R, AWS, MySQL, PostgreS...",株式会社 メンバーズ メンバーズデータアドベンチャーカンパニー,2303,1995-05-31 15:00:00,データエンジニア
21,273394,データエンジニア リーダー候補,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",600,1000,"[SQL, Java, Python, Scala, AWS, MySQL, Postgre...",株式会社 FLINTERS,90,2013-12-31 15:00:00,データエンジニア
23,286281,データエンジニア（データ基盤構築）／リードエンジニア,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",500,650,"[SQL, AWS, Azure, Google Cloud Platform, GCP]",株式会社 エスタイル,63,2006-06-30 15:00:00,データエンジニア
24,267012,データエンジニア,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",600,900,"[C, C++, JavaScript, SQL, Java, Python, Node.j...",株式会社 ヘッドウォータース,290,2005-10-31 15:00:00,データエンジニア
...,...,...,...,...,...,...,...,...,...,...
403,272445,データエンジニア,[],480,800,[],株式会社 識学,227,2015-02-28 15:00:00,データエンジニア
404,211742,データエンジニア（MX統括部）,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",500,1000,"[JavaScript, SQL, Python, AWS, PostgreSQL, Azu...",パーソルビジネスプロセスデザイン 株式会社,18253,1977-08-31 15:00:00,データエンジニア
412,229877,データ基盤エンジニア,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",700,1000,"[JavaScript, Java, Python, AWS, MySQL, Postgre...",アソビュー 株式会社,200,2011-02-28 15:00:00,データエンジニア
421,207594,データベースエンジニア,[SQL],400,600,[SQL],フレックシステムズ 株式会社,56,2003-09-30 15:00:00,データエンジニア


In [31]:
df['job_offer_areas'] = df['job_offer_areas'].apply(ast.literal_eval)
type(df.loc[2, "job_offer_skill_names"])

str

In [32]:
import pandas as pd

# エポック秒のSeries（10桁または13桁の数値）
epoch_series = pd.Series([1401548400, 1580482800, 1209567600])

# pd.to_datetime()で変換
# unit='s' は、入力が秒であることを示します
datetime_series = pd.to_datetime(epoch_series, unit='s')

print(datetime_series)

0   2014-05-31 15:00:00
1   2020-01-31 15:00:00
2   2008-04-30 15:00:00
dtype: datetime64[ns]


In [33]:
import pandas as pd
import numpy as np
import ast

df = pd.read_csv('../data/filtered.csv')

df_copy = df.copy()

df_copy['job_tag'] = 'その他'  # まず'その他'で初期化

df_copy.loc[df_copy['job_offer_name'].str.contains('データ基盤エンジニア|データエンジニア|データベースエンジニア'), 'job_tag'] = 'データエンジニア'

df_copy.loc[df_copy['job_offer_name'].str.contains('データサイエンティスト|データアナリスト'), 'job_tag'] = 'データサイエンティスト'

df_copy.loc[df_copy['job_offer_name'].str.contains('AIエンジニア|機械学習エンジニア|AI開発エンジニア'), 'job_tag'] = 'AIエンジニア'


df_copy.loc[df['job_offer_min_salary']== 0 ,['job_offer_min_salary']].count()


median_salary = df_copy['job_offer_max_salary'].median()
df_copy['job_offer_max_salary'] = df_copy['job_offer_max_salary'].replace(0, np.nan)
df_copy['job_offer_max_salary'] = df_copy['job_offer_max_salary'].fillna(median_salary)

In [34]:
df_copy.loc[df_copy['job_offer_max_salary'] == df_copy['job_offer_max_salary'].min(),:]

,job_offer_id,client_name,job_offer_name,job_tag,job_offer_areas,job_offer_skill_names,employee_count,established,job_offer_min_salary,job_offer_max_salary,avg_salary
92,279618,株式会社 テックエデュケイションカンパニー,AIエンジニア募集(関東),AIエンジニア,"['東京都', '神奈川県', 'フルリモート']","['Python', 'AWS', 'TensorFlow', 'Azure', 'GCP'...",150,2013-02-28 15:00:00,35,80.0,57.5


In [35]:
df_copy.columns.value_counts

<bound method IndexOpsMixin.value_counts of Index(['job_offer_id', 'client_name', 'job_offer_name', 'job_tag',
       'job_offer_areas', 'job_offer_skill_names', 'employee_count',
       'established', 'job_offer_min_salary', 'job_offer_max_salary',
       'avg_salary'],
      dtype='object')>

In [36]:
(df_copy['job_tag'].value_counts()) / (df_copy['job_tag'].value_counts().sum())*100

job_tag
その他            39.635535
AIエンジニア        24.601367
データエンジニア       19.134396
データサイエンティスト    16.628702
Name: count, dtype: float64

In [44]:
df_copy['job_tag'].value_counts().index

Index(['その他', 'AIエンジニア', 'データエンジニア', 'データサイエンティスト'], dtype='object', name='job_tag')